In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
con = sqlite3.connect("D:/Data_Practice/Data_Amzn/final.sqlite")

processed_data = pd.read_sql_query('''SELECT * FROM Reviews''',con)
processed_data.shape
processed_data.head()

final_pos = processed_data[processed_data.Score=='Positive']
final_pos.shape

final_neg = processed_data[processed_data.Score=='Negative']
final_neg.shape

#final_pos_5000 = final_pos[0:5000]
#final_pos_5000.shape
#print(final_pos_5000.head(5))

#final_neg_5000 = final_neg[0:5000]
#final_neg_5000.shape
#print(final_neg_5000.head(5))

final = pd.concat([final_pos , final_neg],axis=0,ignore_index=True)
final.tail()

sample = final.sort_values(by='Time')
score = sample['Score']
sample.head()
sample['Score'].value_counts()

sample_tr,sample_test,sample_try,sample_testy = train_test_split(sample,score,test_size=0.3,random_state=10)

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,2))
final_bow = count_vect.fit_transform(sample_tr['Cleaned_Text'].values)
print(final_bow.shape)
#Standardization
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#std_data = scaler.fit_transform(final_bow.todense())

(254919, 1967430)


In [28]:
final['Text'][138692]

'We have bought most of the flavors in these treats and use them to help train her or just when she needs a little treat. She loves all the flavors so far and they are not crumbly or messy like some other small treats. Will keep purchasing these.'

In [10]:
final['Cleaned_Text'][138692]

b'have bought most the flavor these treat and use them help train her just when she need littl treat she love all the flavor far and they are not crumbl messi like some other small treat will keep purchas these'

In [2]:
final_bow.shape

(254919, 1967430)

In [5]:
x_test=count_vect.transform(sample_test['Cleaned_Text'].values)

In [30]:
type(x_test)

scipy.sparse.csr.csr_matrix

In [69]:
scaler = StandardScaler()
std_data_test = scaler.fit_transform(x_test.todense())
std_data_test.shape

(3000, 12151)

In [31]:
sample_tr['Score'].value_counts()

Positive    214945
Negative     39974
Name: Score, dtype: int64

In [32]:
sample_test['Score'].value_counts()

Positive    92116
Negative    17136
Name: Score, dtype: int64

In [5]:
#Dim Red is optional
from sklearn.manifold import TSNE
model = TSNE(n_components=3,random_state=0)
tsne_data = model.fit_transform(final_dense)
tsne_data.shape

ValueError: 'n_components' should be inferior to 4 for the barnes_hut algorithm as it relies on quad-tree or oct-tree.

In [14]:
type(std_data)
score_1000.shape

(1000,)

In [142]:
sample_try.shape

(7000,)

In [18]:
std_1000 = std_data[0:10000]
score_1000 = score_10000[0:9000]

In [3]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import *
from sklearn.linear_model import LogisticRegression

#data = load_breast cancer() refer: http://scikit-learn.org/stable/modules/

tuned_parameters = [{'C': [10**-4, 0.0001,10**-2,0.1,1,5, 10, 100, 10**3]}]


#Using GridSearchcv

model = GridSearchCV(LogisticRegression(), tuned_parameters, scoring = 'accuracy')

model.fit(final_bow, sample_try)

print(model.best_estimator_) 
print (model.score(final_bow, sample_try)) 

LogisticRegression(C=1)
0.9940373216590368


In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
LRclf = LogisticRegression(C=1,penalty='l2')
LRclf.fit(final_bow,sample_try)

pred = LRclf.predict(x_test)
 
acc = accuracy_score(sample_testy,pred)*100
print(acc)


94.65822135979204


In [7]:
con = confusion_matrix(sample_testy,pred,)
con

array([[13425,  3711],
       [ 2125, 89991]], dtype=int64)

In [25]:
ll= count_vect.get_feature_names()
len(ll)

59090

In [8]:
import pickle as pkl
pkl.dump(LRclf,open("AmznLR1.pkl","wb"))

In [17]:
import pickle as pkl
pkl.dump(LRclf,open("AmznLR.pkl","wb"))

In [9]:
import pickle as pkl
pkl.dump(count_vect,open("BoW1.pkl","wb"))

In [18]:
import pickle as pkl
pkl.dump(count_vect,open("BoW.pkl","wb"))

In [156]:
sample_test[50:51]

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Cleaned_Text
2892,247287,268122,B0000D9NDG,A1UDKKUEP4Z7CA,K. Wong,1,1,Positive,1186358400,Balsamic Convert,"The sweetness is incomparable. Yes, it's $$$,...",b'sweet incompar yes small amount need bottl w...


In [150]:
l=list(sample_test['Cleaned_Text'][50:100])
l

[b'sweet incompar yes small amount need bottl worth last year',
 b'put like five drop hot dog hot admit unbear thought great flavor bad actual think crush habanero hotter good stuff look forward tri ghost pepper',
 b'disappoint receiv brown sand sugar want look like sand decor cupcak beach theme pass lot dark brown sand sugar amazon chose one sinc lighter pictur unfortun arriv realli dark brown',
 b'one favorit movi excit dvd bought sinc great deal arriv quick great job amazon',
 b'like size weight fact clear glass see tea keep warm desk aid coffe cup warmer one cup last sever hour strainer work flawless',
 b'old pomapoo love chew dinosaur safe unabl chew kind bit piec would harm swallow tri nylabon wolf bone abl complet rip piec swallow threw dental dinosaur durabl',
 b'love hot sauc quit collect saw product austin hot sauc shop appli name tear joy snatch differ sauc bought day first one open say disappoint realli sure tast bad label say purifi water soybean oil jalapeno chili cane su

In [163]:
rev=count_vect.transform(l)

In [157]:
rev=count_vect.transform([b'sweet incompar yes small amount need bottl worth last year'])

In [120]:
#std_data_1 = scaler.fit_transform(rev.todense())

In [158]:
print(rev)

  (0, 340)	1
  (0, 1152)	1
  (0, 5294)	1
  (0, 5900)	1
  (0, 6995)	1
  (0, 9698)	1
  (0, 10427)	1
  (0, 11975)	1
  (0, 12048)	1
  (0, 12062)	1


In [164]:
pre=LRclf.predict(rev)

In [165]:
pre

array(['Positive', 'Positive', 'Negative', 'Positive', 'Positive',
       'Negative', 'Negative', 'Negative', 'Negative', 'Positive',
       'Positive', 'Positive', 'Positive', 'Negative', 'Negative',
       'Positive', 'Negative', 'Negative', 'Negative', 'Positive',
       'Negative', 'Negative', 'Negative', 'Positive', 'Negative',
       'Negative', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Positive', 'Positive',
       'Positive', 'Positive', 'Negative', 'Positive', 'Negative',
       'Positive', 'Negative', 'Negative', 'Positive', 'Negative',
       'Negative', 'Negative', 'Positive', 'Positive', 'Negative'],
      dtype=object)

In [169]:
l1=list(sample_test['Text'][50:100])
l1

["The sweetness is incomparable.  Yes, it's $$$, but only a small amount it needed.  A bottle's worth can last more than a year.",
 "I put like five drops on a hot dog and had 2 of them.  It was hot I admit.  But it isn't that unbearable to me.  I thought it has a great flavor and not too bad.  I actually think this crushed habanero I have is hotter.<br />Good stuff!!  Looking forward to trying Ghost pepper",
 'I was disappointed when I received this brown sanding sugar.  I wanted it to look like sand for decorating cupcakes in a beach theme.  I passed by a lot of dark brown sanding sugar on Amazon, and chose this one since it was lighter in the picture.  Unfortunately when it arrived, it was really dark brown.',
 "This is one of my favorite movies!!! I'm so excited to have it on DVD. I bought 2 since it was such a great deal!! It arrived quickly! Great job, amazon!!!",
 "I like it's size, weight, and the fact that it is clear glass so you can see the tea.  I keep it warm at my desk wi